<a href="https://colab.research.google.com/github/strg-frontier/pytorch_wine_quality/blob/master/Poly_0609.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt


In [0]:
from sklearn import datasets
import pandas as pd

In [0]:
from sklearn.base import BaseEstimator
from sklearn.base import BaseEstimator, ClassifierMixin

OUTPUT_SIZE=3

class skNN(BaseEstimator,nn.Module,ClassifierMixin):
    def __init__(self, input_layer=4,hidden_layer1=100,lr=0.01):
        self.input_layer = input_layer
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = int(hidden_layer1 / 2)
        self.lr = lr
        
        super(skNN, self).__init__()
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,OUTPUT_SIZE)
        self.outact = nn.Softmax()
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = self.fc3(x)
        #return self.outact(x)
        return x
        
    #def fit(self, X, y, val_X, val_y, **params):
    def fit(self, X, y, **params):
        self.input_layer = len(X[0])
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        train_scores=[]
        val_scores=[]
        
        tensorX = torch.from_numpy(X).float()
        tensorY = torch.from_numpy(y).long()
        train = TensorDataset(tensorX,tensorY)
        train_loader = DataLoader(train, batch_size=32, shuffle=True)

        for epoch in range(1000):
            total_loss = 0    
            for train_x, train_y in train_loader:
                train_x, train_y = Variable(train_x), Variable(train_y)
                self.optimizer.zero_grad()
                output = self(train_x)
                loss = self.criterion(output, train_y)
                loss.backward()
                self.optimizer.step()
                
                #total_loss += loss.data[0]
            
            if (epoch+1) % 50 == 0:
                train_scores.append(self.score(X,y))
                #val_scores.append(self.score(val_X,val_y))
                print(epoch+1, total_loss)
        return self,train_scores,val_scores
      
    def predict(self, X):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        predicted = self(test_X).data.numpy().argmax(axis=1)
        print(predicted)
        return predicted
    
#    def predict_proba(self, X):
#        tensorX = torch.from_numpy(X).float()
#        test_X = Variable(tensorX)
#        return self(test_X).data[:,1].numpy()
    
    def score(self,X,y):
        predicted = self.predict(X)
        return (predicted==y).sum()/ len(y)
    
    def get_params(self,deep=True):
        return {'hidden_layer1': self.hidden_layer1, 'lr': self.lr}
    
    def set_params(self,**params):
        for parameter, value in params.items():
            setattr(self,parameter, value)
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,OUTPUT_SIZE)
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
        return self


In [0]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier().fit(train_X,train_y)

In [0]:
nn_parameters = {
#    "select__threshold" : ["mean", "median"],
    "clf__hidden_layer": [30,40,50,60],
    "clf__lr": [0.01],
}

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv
!ls

--2019-06-09 05:14:21--  https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2417871 (2.3M) [application/x-httpd-php]
Saving to: ‘Data_for_UCI_named.csv’

Data_for_UCI_named. 100%[===================>]   2.31M  8.30MB/s    in 0.3s    

2019-06-09 05:14:22 (8.30 MB/s) - ‘Data_for_UCI_named.csv’ saved [2417871/2417871]

Data_for_UCI_named.csv	sample_data


In [0]:
data = pd.read_csv('Data_for_UCI_named.csv')

In [5]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [0]:
#ラベルを数値化
y = np.array(data['stabf'].astype('category').cat.codes).astype(float)
X = np.array(data.iloc[:, :-1])
#学習データと検証データを分割
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(
    X, y, test_size = 0.2, random_state=71)

In [12]:
train_X.shape

(8000, 13)

In [0]:
from sklearn.base import BaseEstimator
from sklearn.base import BaseEstimator, ClassifierMixin

train_acc_hist=[]
val_acc_hist=[]

OUTPUT_SIZE=2

class skNN(BaseEstimator,nn.Module,ClassifierMixin):
    def __init__(self, input_layer=10,hidden_layer1=100,hidden_layer2=50,lr=0.01):
        self.input_layer = input_layer
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.lr = lr
        
        super(skNN, self).__init__()
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,OUTPUT_SIZE)
        self.outact = nn.Softmax()
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = self.fc3(x)
        #return self.outact(x)
        return x
        
    #def fit(self, X, y, val_X, val_y, **params):
    def fit(self, X, y, **params):
        self.input_layer = len(X[0])
        print(self.input_layer)
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        train_scores=[]
        val_scores=[]
        
        tensorX = torch.from_numpy(X).float()
        tensorY = torch.from_numpy(y).long()
        train = TensorDataset(tensorX,tensorY)
        train_loader = DataLoader(train, batch_size=32, shuffle=True)

        for epoch in range(100):
            total_loss = 0    
            for train_x, train_y in train_loader:
                train_x, train_y = Variable(train_x), Variable(train_y)
                self.optimizer.zero_grad()
                output = self(train_x)
                loss = self.criterion(output, train_y)
                loss.backward()
                self.optimizer.step()
                
                #total_loss += loss.data[0]
            
            if (epoch+1) % 10 == 0:
                train_acc_hist.append(self.score(X,y))
                val_acc_hist.append(self.score(val_X,val_y))
                print(epoch+1, total_loss)
        return self,train_scores,val_scores
      
    def predict(self, X):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        predicted = self(test_X).data.numpy().argmax(axis=1)
        return predicted
    
#    def predict_proba(self, X):
#        tensorX = torch.from_numpy(X).float()
#        test_X = Variable(tensorX)
#        return self(test_X).data[:,1].numpy()
    
    def score(self,X,y):
        predicted = self.predict(X)
        return (predicted==y).sum()/ len(y)
    
    def get_params(self,deep=True):
        return {'hidden_layer1': self.hidden_layer1,'hidden_layer2':self.hidden_layer2, 'lr': self.lr}
    
    def set_params(self,**params):
        for parameter, value in params.items():
            setattr(self,parameter, value)
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,OUTPUT_SIZE)
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
        return self


In [17]:
model = skNN()
model.fit(train_X,train_y)

10 0
20 0
30 0
40 0
50 0
60 0
70 0
80 0
90 0
100 0


(skNN(hidden_layer1=100, lr=0.01), [], [])

In [9]:
model.score(val_X,val_y)

forwarding


0.7005

In [0]:
nn_parameters = {
#    "select__threshold" : ["mean", "median"],
    "clf__hidden_layer1": [30,40,50],
    "clf__hidden_layer2": [30,40],
    "clf__lr": [0.01],
}

In [30]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline 

select = SelectFromModel(RandomForestClassifier(n_estimators=100, n_jobs=-1))
pca = PCA()
scaler = StandardScaler()
model=skNN()

estimator = [
        #('select',select),
        ('pca',pca),
        ('scaler',scaler),
        ('clf',model)
]

pipe = Pipeline(estimator)

gb_clf = GridSearchCV(pipe, nn_parameters, scoring='accuracy', cv=3, n_jobs=2)

gb_clf.fit(train_X,train_y)

KeyboardInterrupt: ignored

In [26]:
gb_clf.predict(val_X)

array([1, 1, 1, ..., 1, 1, 1])

In [27]:
val_y

array([1., 1., 1., ..., 1., 1., 0.])

In [28]:
gb_clf.score(val_X,val_y)

0.825

(8000, 13)

In [31]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip -d covtype.data.gz	
!ls

--2019-06-09 05:36:39--  https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11240707 (11M) [application/x-httpd-php]
Saving to: ‘covtype.data.gz’

covtype.data.gz     100%[===================>]  10.72M  26.3MB/s    in 0.4s    

2019-06-09 05:36:40 (26.3 MB/s) - ‘covtype.data.gz’ saved [11240707/11240707]

covtype.data  Data_for_UCI_named.csv  sample_data


In [0]:
data = pd.read_csv('covtype.data',header=None).sample(n=5000)

In [33]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
393938,2970,121,16,503,27,2754,246,225,103,2457,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
483827,3294,196,14,437,78,492,216,251,166,735,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2
290979,2936,299,7,42,0,1624,200,237,177,212,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2
95401,2974,171,5,362,37,4001,223,242,154,2311,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
123198,2908,45,2,553,42,3046,219,235,152,2051,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [0]:
#ラベルを数値化
y = np.array(data.iloc[:,-1])
X = np.array(data.iloc[:, :-1])
#学習データと検証データを分割
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(
    X, y, test_size = 0.2, random_state=71)

In [35]:
train_X.shape

(4000, 54)

In [0]:
from sklearn.preprocessing import PolynomialFeatures
interaction = PolynomialFeatures(degree=2,include_bias=False)
train_X_ = interaction.fit_transform(train_X)

In [43]:
train_X_.shape

(4000, 1539)

In [139]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures

select = SelectFromModel(RandomForestClassifier(n_estimators=100, n_jobs=-1))
scaler = StandardScaler()
#pca = PCA()
interaction = PolynomialFeatures(degree=2, include_bias=False)
gb_clf = GradientBoostingClassifier()

gb_parameters = {
    #"select__threshold" : ["mean", "median"],
    "clf__learning_rate": [0.01, 0.05, 0.1],
    "clf__max_depth":[7,9],
    "clf__subsample":[0.5, 1.0],
    "clf__n_estimators":[10]
}

estimator = [
        ('scaler',scaler),
        ('interaction',interaction),
        ('select',select),
        #('pca',pca),
        ('clf',gb_clf)
]

pipe = Pipeline(estimator)
pipe.fit(train_X,train_y)

#gb_clf = GridSearchCV(pipe, gb_parameters, scoring='accuracy', cv=3, n_jobs=2)
#gb_clf.fit(train_X,train_y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('interaction',
                 PolynomialFeatures(degree=2, include_bias=False,
                                    interaction_only=False, order='C')),
                ('select',
                 SelectFromModel(estimator=RandomForestClassifier(bootstrap=True,
                                                                  class_weight=None,
                                                                  criterion='gini',
                                                                  max_depth=None,
                                                                  max_features='auto',
                                                                  max_leaf_nodes=...
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                

In [140]:
pipe.score(val_X,val_y)

0.758

In [47]:
gb_clf.score(val_X,val_y)

0.714

In [138]:
gb_clf = GradientBoostingClassifier().fit(train_X,train_y)
gb_clf.score(val_X,val_y)

0.744

In [0]:
from sklearn.base import BaseEstimator
from sklearn.base import BaseEstimator, ClassifierMixin

OUTPUT_SIZE=8

class skNN(BaseEstimator,nn.Module,ClassifierMixin):
    def __init__(self, input_layer=10,hidden_layer1=300,lr=0.01):
        self.input_layer = input_layer
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = int(self.hidden_layer1 / 2)
        self.hidden_layer3 = int(self.hidden_layer2 / 2)
        self.lr = lr
        
        super(skNN, self).__init__()
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,self.hidden_layer2)
        self.fc4 = nn.Linear(self.hidden_layer2,self.hidden_layer3)
        self.fc5 = nn.Linear(self.hidden_layer3,OUTPUT_SIZE)
        self.outact = nn.Softmax()
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)

    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.dropout(x)
        x = F.relu(self.fc2(x))
        #x = F.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        #return self.outact(x)
        return x
        
    def fit(self, X, y, **params):
        self.input_layer = len(X[0])
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        train_scores=[]
        val_scores=[]
        
        tensorX = torch.from_numpy(X).float()
        tensorY = torch.from_numpy(y).long()
        train = TensorDataset(tensorX,tensorY)
        train_loader = DataLoader(train, batch_size=32, shuffle=True)

        for epoch in range(200):
            total_loss = 0    
            for train_x, train_y in train_loader:
                train_x, train_y = Variable(train_x), Variable(train_y)
                self.optimizer.zero_grad()
                output = self(train_x)
                loss = self.criterion(output, train_y)
                loss.backward()
                self.optimizer.step()
                
                #total_loss += loss.data[0]
            
            if (epoch+1) % 50 == 0:
                train_scores.append(self.score(X,y))
                #val_scores.append(self.score(val_X,val_y))
                print(epoch+1, total_loss)
        return self,train_scores,val_scores
      
    def predict(self, X):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        predicted = self(test_X).data.numpy().argmax(axis=1)
        return predicted
    
    def score(self,X,y):
        predicted = self.predict(X)
        return (predicted==y).sum()/ len(y)
    
    def get_params(self,deep=True):
        return {'hidden_layer1': self.hidden_layer1, 'lr': self.lr}
    
    def set_params(self,**params):
        for parameter, value in params.items():
            print(parameter)
            print(value)
            setattr(self,parameter, value)
        self.hidden_layer2 = int(self.hidden_layer1 / 2)
        self.hidden_layer3 = int(self.hidden_layer2 / 2)
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1,self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2,self.hidden_layer2)
        self.fc4 = nn.Linear(self.hidden_layer2,self.hidden_layer3)
        self.fc5 = nn.Linear(self.hidden_layer3,OUTPUT_SIZE)
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
        return self


In [141]:
select = SelectFromModel(RandomForestClassifier(n_estimators=100, n_jobs=-1))
pca = PCA()
interaction = PolynomialFeatures(degree=2, include_bias=False)
scaler = StandardScaler()
model=skNN()

nn_parameters = {
    "clf__hidden_layer1": [50,100],
    "clf__lr": [0.01,0.05],
}

estimator = [
        ('scaler',scaler),
        ('interaction',interaction),
        ('select',select),
        #('pca',pca),
        ('clf',model)
]

pipe = Pipeline(estimator)
#pipe.fit(train_X,train_y)

nn_clf = GridSearchCV(pipe, nn_parameters, scoring='accuracy', cv=3, n_jobs=-1, verbose=True)

nn_clf.fit(train_X,train_y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.1min finished


hidden_layer1
100
lr
0.01
50 0
100 0
150 0
200 0


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('interaction',
                                        PolynomialFeatures(degree=2,
                                                           include_bias=False,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('select',
                                        SelectFromModel(estimator=RandomForestClassifier(bootstrap=True,
                                                                                         class_weight=None,
                       

In [124]:
train_acc_hist

[0.81875,
 0.81975,
 0.825875,
 0.826,
 0.81675,
 0.827625,
 0.8315,
 0.8245,
 0.82625,
 0.830625]

In [142]:
nn_clf.score(val_X,val_y)

0.654

In [126]:
gb_clf.score(val_X,val_y)

ValueError: ignored

In [57]:
from sklearn.feature_selection import SelectKBest
skb = SelectKBest()
skb.fit(train_X, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [21 28] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SelectKBest(k=10, score_func=<function f_classif at 0x7f2c90e4c510>)

In [0]:
train_X_=skb.transform(train_X)

In [60]:
train_X_.shape

(4000, 10)

In [62]:
train_X.shape

(4000, 54)

In [68]:
select = SelectFromModel(RandomForestClassifier(n_estimators=100, n_jobs=-1))
train_X_ = select.fit_transform(train_X,train_y)
print(train_X_.shape)
skb = SelectKBest()
train_X__ = skb.fit_transform(train_X_,train_y)
print(train_X__.shape)

(4000, 11)
(4000, 10)


In [146]:
nn_clf.best_estimator_.steps[3][1].input_layer

501